In [ ]:
#Libraries to be included
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [ ]:
#Kaggle API command
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

In [ ]:
#Unzipping the file that was downloaded
import zipfile

file_path = '/content/gtzan-dataset-music-genre-classification.zip'

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
#Resizing the images into 180x180 size and converting them to tensor
transform = transforms.Compose([transforms.Resize((180, 180)), transforms.ToTensor()])

In [ ]:
data_path = '/content/Data/images_original'
dataset = datasets.ImageFolder(root=data_path, transform=transform)

In [ ]:
#Splitting the dataset into 70, 20 and 10 for training, validation and testing
train, val = int(len(dataset)*0.7), int(len(dataset)*0.2)
test = int(len(dataset) - train - val)
train_data, val_data, test_data = random_split(dataset, [train, val, test])

In [ ]:
#Loading the dataset
batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
examples = iter(train_loader)
samples, labels = next(examples)
samples.shape, labels.shape #To check the input tensor size

In [ ]:
import matplotlib.pyplot as plt
for i in range(6):
  plt.subplot(2, 3, i+1)
  plt.imshow(samples[i][0], cmap='gray')
plt.show() #Viewing the images

In [ ]:
#Training the model
def train_model(epochs, train_loader, model, loss_function, optimizer):
  for epoch in range(epochs):
    model.train()
    for imgs, labels in train_loader:
      imgs = imgs.cuda()
      labels = labels.cuda()

      #forward
      output = model(imgs)
      loss = loss_fn(output, labels)

      #backward
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    print(f'epoch{epoch+1}/{epochs}, loss = {loss.item():.4f}')
  return model

In [ ]:
#Evaluation of the validation set
def eval1(model):
  correct = 0
  total = 0.0
  with torch.no_grad():
      for images, labels in val_loader:
          images = images.view(images.size(0), -1)  # Flatten the images
          outputs = model(images)
          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  val_accuracy = correct / total * 100
  return val_accuracy

In [ ]:
#Evaluation of the testing set
def eval2(model):
  correct = 0
  total = 0.0
  with torch.no_grad():
      for images, labels in test_loader:
          images = images.view(images.size(0), -1)  # Flatten the images
          outputs = model(images)
          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  test_accuracy = correct / total * 100
  return test_accuracy

In [ ]:
#Convolutional Neural Network with Batch Normalization Layer

class Net3(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_layer1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)
    self.relu = nn.ReLU()
    self.conv_layer2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)
    self.relu_layer = nn.ReLU()
    self.max_pool_layer1 = nn.MaxPool2d(kernel_size = (2,2))

    self.conv_layer3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
    self.relu_layer = nn.ReLU()
    self.conv_layer4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
    self.relu_layer = nn.ReLU()
    self.max_pool_layer2 = nn.MaxPool2d(kernel_size = (2,2))

    self.flatten = nn.Flatten()
    self.fc_layer1 = nn.Linear(in_features = 64 * 42 * 42, out_features = 256)
    self.fc_layer2 = nn.Linear(in_features = 256, out_features = 10)

    #Batch Normalization Layer
    self.batch_norm1 = torch.nn.BatchNorm2d(num_features = 32)
    self.batch_norm2 = torch.nn.BatchNorm2d(num_features = 32)
    self.batch_norm3 = torch.nn.BatchNorm2d(num_features = 64)
    self.batch_norm4 = torch.nn.BatchNorm2d(num_features = 64)

  def forward(self, x):

    x = self.conv_layer1(x)
    x = self.batch_norm1(x)
    x = self.relu(x)
    x = self.conv_layer2(x)
    x = self.batch_norm2(x)
    x = self.relu(x)
    x = self.max_pool_layer1(x)

    x = self.conv_layer3(x)
    x = self.batch_norm3(x)
    x = self.relu(x)
    x = self.conv_layer4(x)
    x = self.batch_norm4(x)
    x = self.relu(x)
    x = self.max_pool_layer1(x)

    x = self.flatten(x)

    x = self.fc_layer1(x)
    x = self.relu(x)
    x = self.fc_layer2(x)

    return x

In [ ]:
model3 = Net3()
model3 = model3.cuda()
loss_fn = nn.CrossEntropyLoss() #Loss function - Cross entropy
optimizer = optim.Adam(model3.parameters()) #Optimizer - Adam

In [ ]:
#For 50 epochs
model3 = train_model(50, train_loader, model3, loss_fn, optimizer)
val_accuracy = eval1(model3)
print(f'Accuracy on Validation set: {val_accuracy}')
test_accuracy = eval2(model3)
print(f'Accuracy on Test set: {test_accuracy}')

In [ ]:
#For 100 epochs
model3 = train_model(100, train_loader, model3, loss_fn, optimizer)
val_accuracy = eval1(model3)
print(f'Accuracy on Validation set: {val_accuracy}')
test_accuracy = eval2(model3)
print(f'Accuracy on Test set: {test_accuracy}')